# Emissions units with Pint

In this notebook we give some examples of how emissions units can be integrated with Pint.

In [1]:
import pint
# from pint.pandas_interface import PintArray  # API still super unstable so don't use yet
from pint.errors import DimensionalityError

from openscm.units import unit_registry

import traceback

Our available units can be shown like so:

In [2]:
dir(unit_registry)

['A',
 'B',
 'BC',
 'BTU',
 'Ba',
 'Bd',
 'Bq',
 'British_horsepower',
 'Btu',
 'C',
 'C12_faraday',
 'C2F6',
 'CCL4',
 'CCl4',
 'CF4',
 'CFC11',
 'CFC113',
 'CFC114',
 'CFC115',
 'CFC12',
 'CH3BR',
 'CH3Br',
 'CH3CCL3',
 'CH3CCl3',
 'CH3CL',
 'CH3Cl',
 'CH4',
 'CO',
 'CO2',
 'Ci',
 'DPI',
 'Da',
 'EC_therm',
 'E_h',
 'Eh',
 'F',
 'Fr',
 'Gy',
 'H',
 'H2O',
 'HALON1202',
 'HALON1211',
 'HALON1301',
 'HALON2402',
 'HCFC141B',
 'HCFC141b',
 'HCFC142B',
 'HCFC142b',
 'HCFC22',
 'HFC125',
 'HFC134A',
 'HFC134a',
 'HFC143A',
 'HFC143a',
 'HFC152A',
 'HFC152a',
 'HFC227EA',
 'HFC227ea',
 'HFC23',
 'HFC236FA',
 'HFC236fa',
 'HFC245FA',
 'HFC245fa',
 'HFC32',
 'HFC365MFC',
 'HFC365mfc',
 'HFC4310',
 'HFC4310MEE',
 'HFC4310mee',
 'Halon1202',
 'Halon1211',
 'Halon1301',
 'Halon2402',
 'Hg',
 'Hz',
 'J',
 'K',
 'KPH',
 'L',
 'Langley',
 'M',
 'MPH',
 'Measurement',
 'N',
 'N2O',
 'N2ON',
 'NH3',
 'NMVOC',
 'NOX',
 'NOx',
 'N_A',
 'Ne',
 'Nm',
 'OC',
 'Oe',
 'P',
 'PPCM',
 'PPI',
 'PSH',
 'Pa',
 

We can see which contexts we have (which can use for e.g. metric conversions).

In [3]:
unit_registry._contexts  # show us which contexts we have available

{'textile': <pint.context.Context at 0x7f53b5563518>,
 'spectroscopy': <pint.context.Context at 0x7f53b5504978>,
 'sp': <pint.context.Context at 0x7f53b5504978>,
 'boltzmann': <pint.context.Context at 0x7f53b5504908>,
 'chemistry': <pint.context.Context at 0x7f53b5504eb8>,
 'chem': <pint.context.Context at 0x7f53b5504eb8>,
 'AR4GWP12': <pint.context.Context at 0x7f53b543f0f0>,
 'CH4_conversions': <pint.context.Context at 0x7f53b54c2198>,
 'NOx_conversions': <pint.context.Context at 0x7f53b54573c8>}

We can now define variables with units.

In [4]:
a = 1 * unit_registry.C
b = 1 * unit_registry.CO2
c = 3 * unit_registry.N2O

They carry units with them.

In [5]:
print(a)
print(b)
print(c)

1 C
1 CO2
3 N2O


We can convert them to base units or to each other.

In [6]:
print(b.to_base_units())
print(b.to('C'))
print(c.to('N'))

0.2727272727272727 C
0.2727272727272727 C
0.9545454545454546 N


Operations are units aware.

In [7]:
print(a + b)
print(a * b)
print((a * b).to_base_units())
print(a / b)
print((a / b).to_base_units())
print(a - b)

1.2727272727272727 C
1 C * CO2
0.2727272727272727 C ** 2
1.0 C / CO2
3.666666666666667 dimensionless
0.7272727272727273 C


If we have proper emissions units, we can convert any bit of the unit we want.

In [8]:
eg1 = 1 * unit_registry.Mt * unit_registry.C / unit_registry.yr
print(eg1)
eg2 = 5 * unit_registry.t * unit_registry.CO2 / unit_registry.s
print(eg2)

1.0 C * megametric_ton / a
5.0 CO2 * metric_ton / second


In [9]:
print(eg1.to("Gt CO2 / day"))
print(eg2.to("Gt C / yr"))

1.0039000638211297e-05 CO2 * gigametric_ton / day
0.04303217178368182 C * gigametric_ton / a


With a context, we can use metric conversion definitions to do our emissions conversions. 
e.g. AR4GWP12 (which is a made up metric where 1C = 20N)
With this metric, we expect 1 CO2 = 12 / 44 C = 12 / 44 \* 20 N = 12 / 44 \* 20 \* 44/14 N2O

In [10]:
with unit_registry.context('AR4GWP12'):
    print(b)
    print(b.to('N2O'))
    print(12 / 44 * 20 * 44 / 14)
    print(a.to('N') + c)  # I am not sure why you need to force the conversion of `a` first...

1 CO2
17.142857142857142 N2O
17.142857142857142
20.954545454545453 N


Without a context to tell us about metrics, if we try to do an invalid conversion, a `DimensionalityError` will be thrown.

In [11]:
try:
    b.to('N2O')
except DimensionalityError:
    traceback.print_exc(limit=1)

Traceback (most recent call last):
  File "<ipython-input-11-10046d0e953b>", line 2, in <module>
    b.to('N2O')
pint.errors.DimensionalityError: Cannot convert from 'CO2' ([carbon]) to 'N2O' ([nitrogen])
